In [16]:
import urllib.parse, webbrowser
import http.server
import socketserver
import threading
import webbrowser
import urllib.parse
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
CLIENT_KEY    = os.getenv("CLIENT_KEY")    # または直書き "sbawd6f9kgtu4fstl5"
CLIENT_SECRET = os.getenv("CLIENT_SECRET") # または直書き "YOUR_CLIENT_SECRET"
REDIRECT_URI  = os.getenv("REDIRECT_URI") 
scopes        = ["user.info.basic"]

params = {
    "client_key": CLIENT_KEY,
    "redirect_uri": REDIRECT_URI,
    "response_type": "code",
    "scope": " ".join(scopes),
    "state": "abc123"
}
auth_url = "https://www.tiktok.com/v2/auth/authorize/?" + urllib.parse.urlencode(params)
print("開くURL:", auth_url)
webbrowser.open(auth_url)   # 既定ブラウザが自動で開く

開くURL: https://www.tiktok.com/v2/auth/authorize/?client_key=sbawd6f9kgtu4fstl5&redirect_uri=https%3A%2F%2Ftiktoker-master.netlify.app%2F.netlify%2Ffunctions%2Fcallback&response_type=code&scope=user.info.basic&state=abc123


True

In [36]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import json
import subprocess
from dotenv import load_dotenv

def load_env():
    load_dotenv()
    client_key = os.getenv("CLIENT_KEY")
    client_secret = os.getenv("CLIENT_SECRET")
    redirect_uri = os.getenv("REDIRECT_URI")
    if not client_key or not client_secret or not redirect_uri:
        print("ERROR: .env に CLIENT_KEY, CLIENT_SECRET, REDIRECT_URI を設定してください。")
        sys.exit(1)
    return client_key, client_secret, redirect_uri

def call_js_and_get_token(code):
    """
    Node.js の callback.js を subprocess で呼び出し、
    認可コード → トークン取得 → JSON を標準出力で受け取る。
    callback.js 側で、
      console.log(JSON.stringify(tokenResponse))
    のように出力していることが前提です。
    """
    try:
        # callback.js のパスを適宜変更してください
        proc = subprocess.run(
            ["node", "functions/callback.js", code],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        print("JS 実行エラー:", e.stderr)
        sys.exit(1)

    try:
        token_json = json.loads(proc.stdout)
    except json.JSONDecodeError:
        print("JS から返された出力が JSON ではありません:")
        print(proc.stdout)
        sys.exit(1)

    return token_json

def save_token(token_json, path="tokens.json"):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(token_json, f, ensure_ascii=False, indent=2)
    print(f"トークン情報を {path} に保存しました。")

def load_and_print_token(path="tokens.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print("\n=== tokens.json の中身 ===")
    print(json.dumps(data, ensure_ascii=False, indent=2))

def main():
    # 環境変数チェック（必要なら callback.js 内でも使えます）
    load_env()

    # 1) 認可コードをユーザ入力で取得
    code = input("認可コード（code）を入力してください: ").strip()
    if not code:
        print("認可コードが入力されませんでした。")
        sys.exit(1)

    # 2) JS を呼び出してトークン取得
    token_json = call_js_and_get_token(code)

    # 3) JSON ファイルに保存
    save_token(token_json)

    # 4) 保存した JSON を読み込んで画面表示
    load_and_print_token()

if __name__ == "__main__":
    main()

認可コード（code）を入力してください:  7LgXjI6Me53RhbdzhtqoQUIRMawgDum81AFBp-TZ38j8bzJ5F9n-cQ_o6QLyaFuyMBKVJAL1y6cq7Ld5LyLfr3ae-MovRSaFSnmT3D-9h8G3LoiwShKCtfp1hSZwNmM-syKPRLhOJDgBdSxX_R-CXicXtmPvR7XFQ2CN6ljL-9f0V1_xU9w8c3iDfqAF4kIZlSb6W2OEjqMT_JdqNbuEa-6vL-GAagtfeLQLVP9tyIM%2A1%215330


JS 実行エラー: node:internal/modules/cjs/loader:1404
  throw err;
  ^

Error: Cannot find module 'C:\Users\ryoij\Desktop\tiktoker\tiktok-oauth\functions\callback.js'
    at Function._resolveFilename (node:internal/modules/cjs/loader:1401:15)
    at defaultResolveImpl (node:internal/modules/cjs/loader:1057:19)
    at resolveForCJSWithHooks (node:internal/modules/cjs/loader:1062:22)
    at Function._load (node:internal/modules/cjs/loader:1211:37)
    at TracingChannel.traceSync (node:diagnostics_channel:322:14)
    at wrapModuleLoad (node:internal/modules/cjs/loader:235:24)
    at Function.executeUserEntryPoint [as runMain] (node:internal/modules/run_main:171:5)
    at node:internal/main/run_main_module:36:49 {
  code: 'MODULE_NOT_FOUND',
  requireStack: []
}

Node.js v22.16.0



AttributeError: 'tuple' object has no attribute 'tb_frame'

In [34]:
import http.server
import socketserver
import threading
import webbrowser
import urllib.parse
import requests
import json
import os
from dotenv import load_dotenv

#── 設定セクション ──────────────────────────────────────

# .env を使う場合はプロジェクト直下に .env を置き、load_dotenv() を有効に
load_dotenv()
CLIENT_KEY    = os.getenv("CLIENT_KEY")    # または直書き "sbawd6f9kgtu4fstl5"
CLIENT_SECRET = os.getenv("CLIENT_SECRET") # または直書き "YOUR_CLIENT_SECRET"
REDIRECT_URI  = os.getenv("REDIRECT_URI")  # "http://localhost:8765/callback"
SCOPES        = ["user.info.basic"]

#── 認可 URL を開く ──────────────────────────────────────

def open_authorize_url():
    params = {
        "client_key":    CLIENT_KEY,
        "redirect_uri":  REDIRECT_URI,
        "response_type": "code",
        "scope":         " ".join(SCOPES),
        "state":         "local"
    }
    url = "https://www.tiktok.com/v2/auth/authorize/?" + urllib.parse.urlencode(params)
    print("ブラウザを開きます。TikTok でログイン＆許可を押してください…")
    webbrowser.open(url, new=1)


#── メインフロー ────────────────────────────────────────

def main():
    # 1) ローカルサーバー起動
    with socketserver.TCPServer(("localhost", PORT), CallbackHandler) as httpd:
        threading.Thread(target=open_authorize_url, daemon=True).start()
        print(f"Listening on http://localhost:{PORT}/callback …")
        

    # 2) code の取得待ち
    if not code_value:
        print("認可コードを取得できませんでした。")
        return
    print("取得した code:", code_value)

    # 3) code → token 交換
    resp = requests.post(
        "https://open.tiktokapis.com/v2/oauth/token/",
        data={
            "client_key":    CLIENT_KEY,
            "client_secret": CLIENT_SECRET,
            "code":          code_value,
            "grant_type":    "authorization_code",
            "redirect_uri":  REDIRECT_URI
        },
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        timeout=15
    )
    token_json = resp.json()
    if resp.status_code != 200:
        print("トークン取得に失敗:", token_json)
        return

    # 4) tokens.json に保存
    with open("tokens.json", "w", encoding="utf-8") as f:
        json.dump(token_json, f, ensure_ascii=False, indent=2)
    print("トークンを tokens.json に保存しました。内容:")
    print(json.dumps(token_json, indent=2, ensure_ascii=False))

    # 2) JS を呼び出してトークン取得
    token_json = call_js_and_get_token(code)

    # 3) JSON ファイルに保存
    save_token(token_json)

    # 4) 保存した JSON を読み込んで画面表示
    load_and_print_token()

if __name__ == "__main__":
    main()


ブラウザを開きます。TikTok でログイン＆許可を押してください…Listening on http://localhost:8765/callback …

認可コードを取得できませんでした。


In [39]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import json
import subprocess

def fetch_token_via_js(code):
    """
    Node スクリプト（functions/callback.js）を subprocess で呼び出し、
    標準出力に出力された JSON を受け取って返す。
    """
    # Jupyter や対話型では __file__ がないので cwd を使う
    script_dir = os.getcwd()

    # callback.js のパス
    js_path = os.path.join(script_dir, 'functions', 'callback.js')
    if not os.path.isfile(js_path):
        print(f"ERROR: callback.js が見つかりません: {js_path}", file=sys.stderr)
        sys.exit(1)

    try:
        proc = subprocess.run(
            ['node', js_path, code],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        print("JS 実行エラー:", e.stderr, file=sys.stderr)
        sys.exit(1)

    try:
        token = json.loads(proc.stdout)
    except json.JSONDecodeError:
        print("JS からの出力が JSON ではありません:", proc.stdout, file=sys.stderr)
        sys.exit(1)

    return token

def main():
    if len(sys.argv) < 2:
        print("Usage: python show_token.py <code>")
        sys.exit(1)

    code = sys.argv[1]
    token = fetch_token_via_js(code)

    # 整形して表示
    print("=== 取得したトークン情報 ===")
    print(json.dumps(token, ensure_ascii=False, indent=2))

if __name__ == '__main__':
    main()

ERROR: callback.js が見つかりません: C:\Users\ryoij\Desktop\tiktoker\tiktok-oauth\functions\callback.js


SystemExit: 1